In [2]:
from mmocr.apis import MMOCRInferencer
ocr = MMOCRInferencer(det=None, rec='Aster')  # 'DBNet'

/home/akarpova/Projects/wagon-number-ocr/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/aster/aster_resnet45_6e_st_mj/aster_resnet45_6e_st_mj-cc56eca4.pth
[{'type': 'InferencerLoader'}, {'type': 'Resize', 'scale': (256, 64)}, {'type': 'PackTextRecogInputs', 'meta_keys': ('img_path', 'ori_shape', 'img_shape', 'valid_ratio')}]
10/14 19:40:11 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.


/home/akarpova/Projects/wagon-number-ocr/.venv/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [16]:
from mmocr.registry import MODELS

In [5]:
preproc_config = {'type': 'STN', 'in_channels': 3, 'resized_image_size': (32, 64), 'output_image_size': (32, 100), 'num_control_points': 20}

In [6]:
stn = MODELS.build(model_config)

In [21]:
ocr.layers()

AttributeError: 'MMOCRInferencer' object has no attribute 'layers'

In [3]:
ocr.textrec_inferencer.model.data_preprocessor

TextRecogDataPreprocessor()

In [15]:
from torch import nn

In [17]:
for k in ocr.textrec_inferencer.model._modules.keys():
    if 'preprocessor' not in k:
        ocr.textrec_inferencer.model._modules[k] = nn.Sequential()

In [18]:
ocr.textrec_inferencer.model._modules

OrderedDict([('data_preprocessor', TextRecogDataPreprocessor()),
             ('preprocessor',
              STN(
                (tps): TPStransform()
                (stn_convnet): Sequential(
                  (0): ConvModule(
                    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (activate): ReLU(inplace=True)
                  )
                  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                  (2): ConvModule(
                    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (activate): ReLU(inplace=True)
                  )
                  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dil

In [ ]:
ocr

In [17]:
import cv2
import torch
from mmengine.dataset import Compose

In [7]:
import pickle

with open('pipeline.pkl', 'rb') as file: 
    # Call load method to deserialze 
    pipeline = pickle.load(file) 

In [122]:
def preprocess(inputs, batch_size: int = 1, **kwargs):
    """Process the inputs into a model-feedable format.

    Args:
        inputs (InputsType): Inputs given by user.
        batch_size (int): batch size. Defaults to 1.

    Yields:
        Any: Data processed by the ``pipeline`` and ``collate_fn``.
    """
    chunked_data = _get_chunk_data(inputs, batch_size)
    yield from map(ocr.textrec_inferencer.collate_fn, chunked_data)

In [136]:
p = list(_get_chunk_data(['EasyOCR/trainer/all_data/train/42030510.jpg']))
p

[[('EasyOCR/trainer/all_data/train/42030510.jpg',
   {'inputs': tensor([[[26, 26, 25,  ...,  7,  6,  6],
             [27, 27, 27,  ...,  7,  6,  6],
             [25, 25, 26,  ...,  7,  7,  7],
             ...,
             [17, 17, 17,  ..., 36, 34, 33],
             [15, 15, 16,  ..., 35, 34, 33],
             [14, 14, 14,  ..., 28, 27, 27]],
    
            [[28, 28, 27,  ...,  7,  6,  6],
             [29, 29, 29,  ...,  7,  6,  6],
             [27, 27, 28,  ...,  7,  7,  7],
             ...,
             [17, 17, 17,  ..., 38, 36, 35],
             [15, 15, 16,  ..., 37, 35, 35],
             [14, 14, 14,  ..., 30, 29, 29]],
    
            [[22, 22, 21,  ...,  7,  6,  6],
             [23, 23, 23,  ...,  7,  6,  6],
             [21, 21, 22,  ...,  7,  7,  7],
             ...,
             [17, 17, 17,  ..., 32, 30, 29],
             [15, 15, 16,  ..., 31, 29, 29],
             [14, 14, 14,  ..., 24, 23, 23]]], dtype=torch.uint8),
    'data_samples': <TextRecogDataSample(


In [22]:
pipeline('EasyOCR/trainer/all_data/train/42030510.jpg')['inputs'].shape

torch.Size([3, 64, 256])

In [19]:
ocr('EasyOCR/trainer/all_data/train/42030510.jpg', show=True, print_result=True)

/home/akarpova/Projects/wagon-number-ocr/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

TypeError: Sequential.forward() takes 2 positional arguments but 3 were given

In [144]:
stn.predict(pipeline('EasyOCR/trainer/all_data/train/42030510.jpg')['inputs'])

AttributeError: 'STN' object has no attribute 'predict'

In [150]:
pre = preprocess(['EasyOCR/trainer/all_data/train/42030510.jpg'])

In [131]:
pre = list(pre)

In [132]:
pre

[]

In [130]:
list(pre)[1]['inputs']

IndexError: list index out of range

In [125]:
res = []
for ri_inputs, data in pre:
    res.append(stn(data))

In [126]:
res

[]

In [105]:
inputs_p = _get_chunk_data('EasyOCR/trainer/all_data/train/42030510.jpg')

In [95]:
h = yield from map(ocr.textrec_inferencer.collate_fn, inputs_p)

SyntaxError: 'yield' outside function (2416207021.py, line 1)

In [151]:
stn(inputs_p)

AttributeError: 'generator' object has no attribute 'dim'

In [23]:
inputs = pipeline('EasyOCR/trainer/all_data/train/42030510.jpg')['inputs']

In [13]:
inputs

tensor([[[26, 26, 25,  ...,  7,  6,  6],
         [27, 27, 27,  ...,  7,  6,  6],
         [25, 25, 26,  ...,  7,  7,  7],
         ...,
         [17, 17, 17,  ..., 36, 34, 33],
         [15, 15, 16,  ..., 35, 34, 33],
         [14, 14, 14,  ..., 28, 27, 27]],

        [[28, 28, 27,  ...,  7,  6,  6],
         [29, 29, 29,  ...,  7,  6,  6],
         [27, 27, 28,  ...,  7,  7,  7],
         ...,
         [17, 17, 17,  ..., 38, 36, 35],
         [15, 15, 16,  ..., 37, 35, 35],
         [14, 14, 14,  ..., 30, 29, 29]],

        [[22, 22, 21,  ...,  7,  6,  6],
         [23, 23, 23,  ...,  7,  6,  6],
         [21, 21, 22,  ...,  7,  7,  7],
         ...,
         [17, 17, 17,  ..., 32, 30, 29],
         [15, 15, 16,  ..., 31, 29, 29],
         [14, 14, 14,  ..., 24, 23, 23]]], dtype=torch.uint8)

In [167]:
stn

STN(
  (tps): TPStransform()
  (stn_convnet): Sequential(
    (0): ConvModule(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ConvModule(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ConvModule(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (5): MaxPool2d(kernel_siz

In [11]:
tf = Compose([{"type": "InferencerLoader"}, {"type": "Resize", "scale": [256, 64]}, {"type": "PackTextRecogInputs", "meta_keys": ["img_path", "ori_shape", "img_shape", "valid_ratio"]}])

[{'type': 'InferencerLoader'}, {'type': 'Resize', 'scale': [256, 64]}, {'type': 'PackTextRecogInputs', 'meta_keys': ['img_path', 'ori_shape', 'img_shape', 'valid_ratio']}]


KeyError: 'InferencerLoader is not in the mmengine::transform registry. Please check whether the value of `InferencerLoader` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'

In [25]:
img_tensor = tf('EasyOCR/trainer/all_data/train/42030510.jpg')['inputs']

In [27]:
stn(img_tensor)

NameError: name 'resize_img' is not defined